# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np
from ddpg_agent import Agent
from collections import deque
import matplotlib.pyplot as plt
import random
import torch 

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [2]:
env = UnityEnvironment(file_name='Reacher_Windows_x86_64\Reacher.exe')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726624e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 3. Train DDPG
Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [ ]:
def ddpg(n_episodes=2000, max_t=1000,  maxlen = 100, solved_score = 30.0, train_mode=True):
    """Deep Deterministic Policy Gradient (DDPG) 
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        train_mode (bool)     : if 'True' set environment to training mode
        maxlen(int): the window size to take the average scores 
        solved_score (float)  : minimun average score over maxlen consecutive episodes, and over all agents
    """
   
    scores_window = deque(maxlen = maxlen)  # mean score list over maxlen episodes
    scores_all = []  # all scores for each agent and each episode
    mean_scores_over_agents = [] # the mean score averaged over agents for each episode
    mean_scores_over_agents_episodes = [] # the mean score averaged over agents and episodes
   
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode = train_mode)[brain_name] # reset the environment
        states = env_info.vector_observations     # get the current state for each agent
        scores = np.zeros(num_agents)               # scores for each agent
        for t in range(max_t):
            actions = agent.act(states, add_noise=True)  # select the action with adding noise, for each agent
            env_info = env.step(actions)[brain_name]     # send the action to the environment
            next_states = env_info.vector_observations   # get the next state for each agent
            rewards = env_info.rewards                   # get the reward for each agent
            dones = env_info.local_done                  # see if episode has finished
            agent.step(states, actions, rewards, next_states, dones)
            states = next_states
            scores += rewards            
            if np.any(dones):
                break 
        
        # save the scores for each agent and the mean score averaged over all agents for this episode
        scores_all.append(scores)     
        mean_scores_over_agents.append(np.mean(scores))
        
        # check the mean score that averaged over all agents and in continous episodes in a window
        scores_window.append(np.mean(scores))   
        average_score = np.mean(scores_window) 
        
        # save the mean score averaged over agents and episodes
        mean_scores_over_agents_episodes.append(average_score)
        
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, average_score))
        #torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
        #torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
        if average_score>=solved_score and train_mode:
            print('\n Target score reached in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode,average_score))
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor_sigma_point2.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic_sigma_point2.pth')
            break
    return scores_all,  mean_scores_over_agents, mean_scores_over_agents_episodes

### 4. Plot the training scores

In [ ]:
# Train the DDPG 
agent = Agent(state_size=state_size, action_size=action_size, random_seed = 1234)
scores,  mean_scores_over_agents, mean_scores_over_agents_episodes = ddpg()

C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\functional.py:1320: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Episode 1	Average Score: 0.40
Episode 2	Average Score: 0.45
Episode 3	Average Score: 0.45
Episode 4	Average Score: 0.52
Episode 5	Average Score: 0.58
Episode 6	Average Score: 0.60
Episode 7	Average Score: 0.67
Episode 8	Average Score: 0.84
Episode 9	Average Score: 0.92
Episode 10	Average Score: 0.96
Episode 11	Average Score: 1.00
Episode 12	Average Score: 1.04
Episode 13	Average Score: 1.07
Episode 14	Average Score: 1.12
Episode 15	Average Score: 1.14
Episode 16	Average Score: 1.15
Episode 17	Average Score: 1.21
Episode 18	Average Score: 1.28
Episode 19	Average Score: 1.38
Episode 20	Average Score: 1.44
Episode 21	Average Score: 1.57
Episode 22	Average Score: 1.63
Episode 23	Average Score: 1.73
Episode 24	Average Score: 1.78
Episode 25	Average Score: 1.81
Episode 26	Average Score: 1.86
Episode 27	Average Score: 1.98
Episode 28	Average Score: 2.04
Episode 29	Average Score: 2.11
Episode 30	Average Score: 2.16
Episode 31	Average Score: 2.25
Episode 32	Average Score: 2.33
Episode 33	Averag

In [ ]:
plt.figure(figsize=(20,10))
scores=np.array(scores)
m,n=scores.shape
plt.plot(np.arange(1, len(scores)+1), mean_scores_over_agents_episodes,'r')
plt.plot(np.arange(1, len(scores)+1), mean_scores_over_agents, 'g')
for i in range(n):
    plt.plot(np.arange(1, len(scores)+1), scores[:,i], 'b*')    
plt.grid()
plt.xlabel('Episode')
plt.ylabel('Training scores')
plt.legend(( 'Mean score averaged over 20 agents and 100 episodes ', 'Mean score averaged over 20 agents','Scores for all agents'), loc='upper right')
plt.show()

 ### 5. Watch the smart agent to play

In [ ]:
agent.actor_local.load_state_dict(torch.load('checkpoint_actor.pth'))
agent.critic_local.load_state_dict(torch.load('checkpoint_critic.pth'))

env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = agent.act(states)                        # select an action (for each agent)
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('score averaged over all 20 agents {}'.format(np.mean(scores)))

### 6. When finished, you can close the environment.

In [ ]:
# env.close()